## Lab: Deep Learning

#### A single Layer Network on the Hitters Data

In [9]:
import pandas as pd
import numpy as np
import random
import math
import torch
from torch.utils import data

Load data

In [44]:
Gitters = pd.read_csv('Gitters.csv')
n = Gitters.shape[0]
random.seed(13)
ntest = math.trunc(n / 3)
testid = random.sample(range(n), ntest)
trainid = [i for i in range(n) if i not in testid]
x_train = torch.from_numpy(Gitters.iloc[trainid,:-1].to_numpy()).float()
y_train = torch.from_numpy(Gitters.iloc[trainid,-1].to_numpy().reshape(-1, 1)).float()
x_test = torch.from_numpy(Gitters.iloc[testid,:-1].to_numpy()).float()
y_test = torch.from_numpy(Gitters.iloc[testid,-1].to_numpy().reshape(-1, 1)).float()

In [45]:
y_train = (y_train - y_train.mean()) / np.sqrt(y_train.var())
y_test = (y_test - y_test.mean()) / np.sqrt(y_test.var())

In [56]:
def load_array(data_arrays, batch_size, is_train=True):
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)

batch_size = 32
epochs = 20
data_iter = load_array((x_train, y_train), batch_size)

Build network

In [57]:
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

class Net(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, 1)
        self.dropout = nn.Dropout(0.4)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

Set parameters

In [58]:
net = Net(20)
loss_function = nn.MSELoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)

Training

In [59]:
losses = []
for epoch in range(epochs):
    loss_epoch = 0
    for x,y in data_iter:
        optimizer.zero_grad()
        loss = loss_function(net(x), y)
        loss.backward()
        optimizer.step()
        loss_epoch += loss.item()
    print(f'epoch {epoch + 1}, loss {loss_epoch:f}')

epoch 1, loss 7.140122
epoch 2, loss 5.916814
epoch 3, loss 4.766182
epoch 4, loss 4.590686
epoch 5, loss 4.455988
epoch 6, loss 4.036636
epoch 7, loss 3.968684
epoch 8, loss 4.011201
epoch 9, loss 4.137891
epoch 10, loss 3.561571
epoch 11, loss 4.553437
epoch 12, loss 3.916849
epoch 13, loss 3.826640
epoch 14, loss 3.985428
epoch 15, loss 3.309112
epoch 16, loss 3.431372
epoch 17, loss 3.041155
epoch 18, loss 3.340093
epoch 19, loss 3.124849
epoch 20, loss 3.648586


Testing

In [69]:
pred = net(x_test)
loss_function(pred, y_test).item()

0.5584160685539246